# Create a very basic code assistant and experiment with some prompting

In [1]:
import json
import os
import openai

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [94]:
from endpoint_utils import openai_chat_completions_create

In [3]:
client = openai.Client()

In [20]:
model_list = client.models.list().data
model_list = [m for m in model_list if m.id.startswith("gpt-4")]
model_list

[Model(id='gpt-4-0613', created=1686588896, object='model', owned_by='openai'),
 Model(id='gpt-4', created=1687882411, object='model', owned_by='openai'),
 Model(id='gpt-4-vision-preview', created=1698894917, object='model', owned_by='system'),
 Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'),
 Model(id='gpt-4-0314', created=1687882410, object='model', owned_by='openai')]

In [28]:
OPEN_AI_MODEL = "gpt-4-1106-preview"

In [99]:
# function to run python code from a string using the ast module
def run_python_code(code, globals_=None, locals_=None):
    import ast
    if globals_ is None:
        globals_ = {}
    if locals_ is None:
        locals_ = {}
    parsed = ast.parse(code)
    code_obj = compile(parsed, filename="<ast>", mode="exec")
    exec(code_obj, globals_, locals_)

In [105]:
# messages = [
#     {"role": "system", "content": """\
# You are a helpful and highly expert python coding assistant. \
# When asked to generate code you always give the best code possible within code blocks surrounding the python code.
# When providing example usage, this should be in a second code block, with no further text between each code block.
# """}
# ]

messages = [
    {"role": "system", "content": """\
You are a helpful and highly expert python coding assistant. \
When asked to generate code you always give the best code possible within code blocks surrounding the python code.
"""},
]

In [106]:
prompt = """\
Write python code that will download a file from a URL available as an existing local variable `url` and save it to a file. \
The `url` variable should not be set by the code you generate. \
"""

In [107]:
messages_req = messages + [{"role": "user", "content": prompt}]
content, response = openai_chat_completions_create(messages_req, model=OPEN_AI_MODEL, client=client, return_response=True, max_retries=2)
completion_message = response.choices[0].message.model_dump(exclude_unset=True)

Setup local variables to use. Prompt should ask to use these, not set examples

In [108]:
# locals_ = {"url": "https://cf1.zzounds.com/media/productmedia/fit,600by600/quality,85/Prophet-5_Left_Angle_820175-435c1c5a0e3d3a898c11179315824fc9.jpg"}
url = "https://cf1.zzounds.com/media/productmedia/fit,600by600/quality,85/Prophet-5_Left_Angle_820175-435c1c5a0e3d3a898c11179315824fc9.jpg"

In [110]:
content = completion_message["content"]
assert '```python' in content, "No code block found in response\n\n" + content

code = content.split('```python')[1].split('```')[0]
print('Code found\n---\n' + code, flush=True)

run_it = input('Run code? [y/N]: ')
if run_it.lower() == 'y':
    run_python_code(code, globals_=globals(), locals_=locals())

Code found
---

import requests

# Assume the 'url' variable is already defined
# url = 'http://example.com/somefile.txt'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Extract the filename from the URL, assuming it is the last segment after '/'
    filename = url.split('/')[-1]

    # Write the content of the request to a file
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"File '{filename}' has been downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File 'Prophet-5_Left_Angle_820175-435c1c5a0e3d3a898c11179315824fc9.jpg' has been downloaded successfully.
